In [2]:
import pandas as pd
import numpy as np

In [3]:
# 如下所示,已知A, B两个DataFrame表示的是与用户相关的行为数据，其中id表示用户, f1-f8表示不同行为，channel表示不同的入口。例如：
# A中第一行数据,表示用户"1"在入口"a"下的不同行为得分。
A = pd.DataFrame([["1", 2, 3, None, 4, 5, "a"], 
                  ["1", 3, 4, 6, 5, 6, "b"], 
                  ["2", None, 0, None, 6, None, "a"], 
                  ["2", 3, 0, None, 7, None, "b"],
                  ["2", 3, None, 4, None, 7, "c"],
                  ["3", 6, 9, 0, None, 4, "a"],
                  ["3", 2, 6, None, 0, None, "b"],
                  ["3", 3, 4, 7, None, 8, "c"]], 
                  columns=["id", "f1", "f2", "f3", "f4",  "f5", "channel"])

B = pd.DataFrame([["1", 1, 3, 9, "c"], 
                  ["2", 3, 4, 8, "c"], 
                  ["3", 4, 5, 7, "c"], 
                  ["4", 1, 2, 0, "d"]], 
                  columns=["id", "f6", "f7", "f8", "channel"])

In [4]:
A

,id,f1,f2,f3,f4,f5,channel
0,1,2.0,3.0,NaN,4.0,5.0,a
1,1,3.0,4.0,6.0,5.0,6.0,b
2,2,NaN,0.0,NaN,6.0,NaN,a
3,2,3.0,0.0,NaN,7.0,NaN,b
4,2,3.0,NaN,4.0,NaN,7.0,c
5,3,6.0,9.0,0.0,NaN,4.0,a
6,3,2.0,6.0,NaN,0.0,NaN,b
7,3,3.0,4.0,7.0,NaN,8.0,c


In [5]:
B

,id,f6,f7,f8,channel
0,1,1,3,9,c
1,2,3,4,8,c
2,3,4,5,7,c
3,4,1,2,0,d


In [8]:
# 问题1，异常情况检测，检测A中是否存在缺失值 


,id,f1,f2,f3,f4,f5,channel
0,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False
2,False,True,False,True,False,True,False
3,False,False,False,True,False,True,False
4,False,False,True,False,True,False,False
5,False,False,False,False,True,False,False
6,False,False,False,True,False,True,False
7,False,False,False,False,True,False,False


In [9]:
A

,id,f1,f2,f3,f4,f5,channel
0,1,2.0,3.0,NaN,4.0,5.0,a
1,1,3.0,4.0,6.0,5.0,6.0,b
2,2,NaN,0.0,NaN,6.0,NaN,a
3,2,3.0,0.0,NaN,7.0,NaN,b
4,2,3.0,NaN,4.0,NaN,7.0,c
5,3,6.0,9.0,0.0,NaN,4.0,a
6,3,2.0,6.0,NaN,0.0,NaN,b
7,3,3.0,4.0,7.0,NaN,8.0,c


In [99]:
# 问题2，对A中的缺失值处理，
# 处理规则如下：
#     1, f4, f5列的缺失值采用f4, f5列的平均值替代； 
#     2, f1,f2,f3列缺失值处理方式：先确定那一行没有缺失值，假设第2行没有缺失值，则取第2行的最大值替代f1, f2, f3列的缺失值；
# 得到如下结果A：


,id,f1,f2,f3,f4,f5,channel
0,1,2.0,3.0,6.0,4.0,5.0,a
1,1,3.0,4.0,6.0,5.0,6.0,b
2,2,6.0,0.0,6.0,6.0,6.0,a
3,2,3.0,0.0,6.0,7.0,6.0,b
4,2,3.0,6.0,4.0,4.4,7.0,c
5,3,6.0,9.0,0.0,4.4,4.0,a
6,3,2.0,6.0,6.0,0.0,6.0,b
7,3,3.0,4.0,7.0,4.4,8.0,c


In [75]:
# 问题3，在问题2的基础上对A进行处理，得到如下结果A

,f1,f2,f3,f4,f5
id,,,,,
1,5.0,7.0,12.0,9.0,11.0
2,12.0,6.0,16.0,17.4,19.0
3,11.0,19.0,13.0,8.8,18.0


In [103]:
# 问题4，在问题3的基础上，将A, B按"id"进行合并，得到如下结果C


,f1,f2,f3,f4,f5,f6,f7,f8
id,,,,,,,,
1,5.0,7.0,12.0,9.0,11.0,1,3,9
2,12.0,6.0,16.0,17.4,19.0,3,4,8
3,11.0,19.0,13.0,8.8,18.0,4,5,7
4,0.0,0.0,0.0,0.0,0.0,1,2,0


In [101]:
# 问题5，在问题4的基础上对C增加一列("f9")，其中f9 = 2*f1-f2+f4


,f1,f2,f3,f4,f5,f6,f7,f8,f9
id,,,,,,,,,
1,5.0,7.0,12.0,9.0,11.0,1,3,9,12.0
2,12.0,6.0,16.0,17.4,19.0,3,4,8,35.4
3,11.0,19.0,13.0,8.8,18.0,4,5,7,11.8
4,0.0,0.0,0.0,0.0,0.0,1,2,0,0.0


In [104]:
# 问题6， 在问题4的基础上进行特征处理.对C进行处理，处理规则如下：每个用户(每一行)的特征进行最大-最小值标准化


,f1,f2,f3,f4,f5,f6,f7,f8
id,,,,,,,,
1,0.360360,0.540541,0.990991,0.720721,0.900901,0.00000,0.180180,0.720721
2,0.559006,0.186335,0.807453,0.894410,0.993789,0.00000,0.062112,0.310559
3,0.463576,0.993377,0.596026,0.317881,0.927152,0.00000,0.066225,0.198675
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.47619,0.952381,0.000000
